installs

In [1]:
%%script echo skipping
%pip install snowflake-connector-python
%pip install pyarrow
%pip install pandas
!pip3 install confluent-kafka

skipping


imports

In [2]:
from kafka_helpers import *
from snowflake_helpers import *
from transformation_helpers import *


/opt/homebrew/lib/python3.9/site-packages/snowflake/connector/options.py:96: UserWarning: You have an incompatible version of 'pyarrow' installed (9.0.0), please install a version that adheres to: 'pyarrow<8.1.0,>=8.0.0; extra == "pandas"'
  warn_incompatible_dep(


### **SETUP KAFKA**

In [4]:
consumer = connect_to_kafka_consumer()
data = stream_kafka_topic(consumer, KAFKA_TOPIC_NAME, 2000)

In [5]:
data

['No message available',
 'No message available',
 '--------- beginning of main\n',
 '--------- beginning of a new ride\n',
 '2022-07-06 23:29:35.435798 mendoza v9: Getting user data from server..\n',
 '2022-07-06 23:29:35.935822 mendoza v9: [SYSTEM] data = {"user_id":100,"name":"Steven Pearce","gender":"male","address":"0 Simpson locks,Wilkinsonshire,E68 8SH","date_of_birth":273715200000,"email_address":"steven.p@gmail.com","height_cm":167,"weight_kg":83,"account_create_date":1647993600000,"bike_serial":"SN0000","original_source":"direct"}\n',
 '2022-07-06 23:29:36.435842 mendoza v9: [INFO]: Ride - duration = 1.0; resistance = 0\n',
 '2022-07-06 23:29:36.935860 mendoza v9: [INFO]: Telemetry - hrt = 0; rpm = 0; power = 0.0\n',
 '2022-07-06 23:29:37.435875 mendoza v9: [INFO]: Ride - duration = 2.0; resistance = 0\n',
 '2022-07-06 23:29:37.935885 mendoza v9: [INFO]: Telemetry - hrt = 0; rpm = 0; power = 0.0\n',
 '2022-07-06 23:29:38.435896 mendoza v9: [INFO]: Ride - duration = 3.0; resis

### **PROCESS KAFKA W PANDAS**

In [6]:
df = init_df(data)
df

In [8]:
df = format_df(df)
df

In [10]:
df.dtypes

log                            object
is_new_ride                      bool
is_info                          bool
is_system                        bool
date                   datetime64[ns]
heart_rate                    float64
resistance                    float64
rpm                           float64
power                         float64
user_id                       float64
name                           object
gender                         object
address                        object
date_of_birth                 float64
email_address                  object
height_cm                     float64
weight_kg                     float64
account_create_date           float64
bike_serial                    object
original_source                object
dtype: object

### **SETUP SNOWFLAKE**

connection, cursor, schemas

In [11]:
conn = connect_to_snowflake()
cs = conn.cursor()
show_schemas(cs)

[(datetime.datetime(2022, 10, 4, 9, 47, 20, 358000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>),
  'INFORMATION_SCHEMA',
  'N',
  'N',
  'DELATON',
  '',
  'Views describing the contents of schemas in this database',
  '',
  '1'),
 (datetime.datetime(2022, 9, 29, 1, 9, 6, 401000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>),
  'PUBLIC',
  'N',
  'N',
  'DELATON',
  'SYSADMIN',
  '',
  '',
  '1'),
 (datetime.datetime(2022, 9, 29, 1, 9, 44, 900000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>),
  'TEC_BIKE_PRODUCTION',
  'N',
  'N',
  'DELATON',
  'SYSADMIN',
  '',
  '',
  '1'),
 (datetime.datetime(2022, 10, 4, 6, 15, 58, 74000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>),
  'TEC_BIKE_STAGING',
  'N',
  'N',
  'DELATON',
  'SYSADMIN',
  '',
  '',
  '1'),
 (datetime.datetime(2022, 10, 4, 4, 39, 6, 77000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>),
  'YUSRA_STORIES_STAGING',
  'N',
 

Ensure cursor is set to use right schema

In [12]:
create_staging_schema(cs)
use_staging_schema(cs)

Check tables in schema

In [13]:
show_tables(cs, STAGING_SCHEMA)

[(datetime.datetime(2022, 10, 4, 4, 39, 6, 220000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>),
  'TEST_TABLE',
  'DELATON',
  'YUSRA_STORIES_STAGING',
  'TABLE',
  '',
  '',
  2,
  1024,
  'SYSADMIN',
  '1',
  'OFF',
  'OFF',
  'N')]

Check test df

In [14]:
test_df = fetch_test_data(cs)
test_df

,COL1,COL2
0,123,test string1
1,456,test string2


### **PUTTING DF INTO SNOWFLAKE**

WRITE_PANDAS WAY 

In [15]:
from snowflake.connector.pandas_tools import write_pandas

In [47]:
df.columns.values

array(['log', 'is_new_ride', 'is_info', 'is_system', 'date', 'heart_rate',
       'resistance', 'rpm', 'power', 'user_id', 'name', 'gender',
       'address', 'date_of_birth', 'email_address', 'height_cm',
       'weight_kg', 'account_create_date', 'bike_serial',
       'original_source'], dtype=object)

In [41]:
cs.execute(f"""
        CREATE OR REPLACE TABLE df_test_table
        ("log" STRING,
        "is_new_ride" BOOLEAN,
        "is_info" BOOLEAN,
        "is_system" BOOLEAN,
        "date" DATETIME,
        "heart_rate" INTEGER,
        "resistance" INTEGER,
        "rpm" INTEGER,
        "power" FLOAT,
        "user_id" STRING,
        "name" STRING,
        "gender" STRING,
        "address" STRING,
        "date_of_birth" FLOAT,
        "email_address" STRING,
        "height_cm" INTEGER,
        "weight_kg" INTEGER,
        "account_create_date" FLOAT,
        "bike_serial" STRING,
        "original_source" STRING
        )
""")

In [42]:
show_tables(cs, STAGING_SCHEMA)

[(datetime.datetime(2022, 10, 4, 9, 57, 13, 363000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>),
  'DF_TEST_TABLE',
  'DELATON',
  'YUSRA_STORIES_STAGING',
  'TABLE',
  '',
  '',
  0,
  0,
  'SYSADMIN',
  '1',
  'OFF',
  'OFF',
  'N'),
 (datetime.datetime(2022, 10, 4, 4, 39, 6, 220000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>),
  'TEST_TABLE',
  'DELATON',
  'YUSRA_STORIES_STAGING',
  'TABLE',
  '',
  '',
  2,
  1024,
  'SYSADMIN',
  '1',
  'OFF',
  'OFF',
  'N')]

In [43]:
write_pandas(conn, df, 'DF_TEST_TABLE')

(True,
 1,
 2001,
 [('zusaa/file0.txt', 'LOADED', 2001, 2001, 1, 0, None, None, None, None)])

**WORKING !** 

In [44]:
cs.execute(f'SELECT * FROM DF_TEST_TABLE WHERE "is_system" = TRUE').fetch_pandas_all()

,log,is_new_ride,is_info,is_system,date,heart_rate,resistance,rpm,power,user_id,name,gender,address,date_of_birth,email_address,height_cm,weight_kg,account_create_date,bike_serial,original_source
0,2022-07-06 23:29:35.935822 mendoza v9: [SYSTEM...,False,False,True,"{'epoch': 1657150175910622, 'fraction': 0}",NaN,NaN,NaN,NaN,100,Steven Pearce,male,"0 Simpson locks,Wilkinsonshire,E68 8SH",2.737152e+11,steven.p@gmail.com,167,83,1.647994e+12,SN0000,direct
1,2022-07-06 23:38:15.945298 mendoza v9: [SYSTEM...,False,False,True,"{'epoch': 1657150695916498, 'fraction': 0}",NaN,NaN,NaN,NaN,100,Steven Pearce,male,"0 Simpson locks,Wilkinsonshire,E68 8SH",2.737152e+11,steven.p@gmail.com,167,83,1.647994e+12,SN0000,direct
